# Stochastic Gradient Descent

In [ ]:
# pip install packages that are not in Pyodide
%pip install ipympl==0.9.3
%pip install seaborn==0.12.2

In [ ]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mude_tools import magicplotter
from cycler import cycler
import seaborn as sns

%matplotlib widget

# Set the color scheme
sns.set_theme()
colors = [
    "#0076C2",
    "#EC6842",
    "#A50034",
    "#009B77",
    "#FFB81C",
    "#E03C31",
    "#6CC24A",
    "#EF60A3",
    "#0C2340",
    "#00B8C8",
    "#6F1D77",
]
plt.rcParams["axes.prop_cycle"] = cycler(color=colors)

This page contains interactive python element: click {fa}`rocket` --> {guilabel}`Live Code` in the top right corner to activate it.

## Introduction

<iframe width="560" height="315" src="https://www.youtube.com/embed/Xy_1cRy9AOU?si=a0SnL6hmAtQHNyOU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

For now, we used the same dataset at once. In some situations, it might be beneficial or necessary to look at only a part of the dataset, e.g., when

- $N$ is too large and computing $( \boldsymbol{\Phi}^T \boldsymbol{\Phi} )^{-1}$ becomes prohibitively expensive
- the model is nonlinear in $\mathbf{w}$, and $\mathbf{w}_\mathrm{ML}$ does not have a closed-form solution
- the dataset is arriving sequentially (e.g., in real-time from a sensor)

Instead of solving for $\mathbf{w}$ directly, we could employ an iterative optimization strategy. Let's first take a look at the data part of the error function, and its gradient with respect to $\mathbf{w}$:

$$
E_D = \frac{1}{2N} \sum_{n = 1}^{N} \left( t_n - \mathbf{w}^T \boldsymbol{\phi}_n \right)^2 \quad \mathrm{with \, gradient} \quad \nabla E_D = - \frac{1}{N} \sum_{n=1}^N \left(t_n - \mathbf{w}^T \boldsymbol{\phi}_n \right) \boldsymbol{\phi}_n.
$$

The standard formulation does not include the division by the dataset size, the stepsize is purely regulated through the learning rate. However, normalizing the gradient with $N$ makes the influence of the learning rate more consistent when considering different datasets. With a standard gradient descent algorithm, the update rule for the weights is given by

$$
\mathbf{w}^{(\tau + 1)} = \mathbf{w}^{(\tau)} - \eta \nabla E_D
$$

with a fixed *learning rate* $\eta$. The costs for the gradient computations are independent of the dataset size $N$, when only considering subset $\mathcal{B}$ of our dataset with $N_{\mathcal{B}}$ data points. If we pick a random subset $\mathcal{B}$ for each iteration of the optimization scheme, we have derived the *stochastic gradient descent* algorithm. Together with its numerous variants, this algorithm forms the backbone of many machine learning techniques. Most deep learning libraries, such as [`TensorFlow`](https://www.tensorflow.org/) or [`PyTorch`](https://pytorch.org/) offer implementations of these algorithms.

We looked at the unregularized model to introduce SGD, but the extension to the regularized model is straightforward. Remember, in this case, the objective function is given by

$$
E (\mathbf{w}) = \frac{1}{2N} \sum_{n = 1}^{N} \left( t_n - \mathbf{w}^T \boldsymbol{\phi}_n \right)^2 + \frac{\lambda}{2N} \mathbf{w}^T \mathbf{w},
$$

and its gradient with respect to $\mathbf{w}$ reads

$$
\nabla E = \frac{1}{N} \left( - \sum_{n=1}^N \left(t_n - \mathbf{w}^T \boldsymbol{\phi}_n \right) \boldsymbol{\phi}_n + \lambda \, \mathbf{w} \right).
$$

When looking at the expresssion in the outer bracket, it becomes clear there are **two competing terms**: the first term pulls the weights towards the data, and the second term pulls them towards zero. Looking at the gradient, you can also see why ridge regression often is referred to as weight decay. The larger the weights become, the stronger the regularization term pulls them towards zero. If the data would not support a value for a certain weight, its presence in the gradient will lead to the weight decaying at a rate proportional to its magnitude.

In [ ]:
# This function returns the gradient of the cost function
def get_gradient(x, t, w, basis, lam=0.0, **kwargs):
    # Get the variable matrix using the basis function phi
    Phi = basis(x.reshape(-1), **kwargs)
    t = t.reshape(-1)

    return (-(t - w @ Phi.T) @ Phi + lam * w) / len(t)


# this computes the rmse
def get_rmse(t_1, t_2, scaler):
    se = (
        scaler.inverse_transform(t_1[:, None]) - scaler.inverse_transform(t_2[:, None])
    ) ** 2
    rmse = np.sqrt(sum(se) / len(t_1))
    return rmse

In [ ]:
# set parameters
N, N_train = 500, 20
M_radial = 50  # number of radial basis functions
l_radial = 0.20  # lenghtscale of radial basis functions
eta = 0.01  # learning rate
epochs = 1000  # number of epochs
N_batch = 5  # size of the minibatch B
lam = np.exp(0.05)  # Regularization parameter

# select where to plot solution
epoch_plot_marks = np.logspace(np.log10(5), np.log10(epochs), 10, dtype=int)

# randomly init weights
np.random.seed(6)
w = (np.random.rand(M_radial) - 0.5) / M_radial
w_reg = w.copy()

# get colormap
rgb = cm.get_cmap("summer")(np.linspace(0, 1, 10))[:, :3]

# generate, scale, and partition data
x, t = f_data(N=N + N_train)
x_plot = np.linspace(0, 2 * np.pi, 100)
xscaler, tscaler = StandardScaler(), StandardScaler()
x_sc, t_sc = xscaler.fit_transform(x[:, None]), tscaler.fit_transform(t[:, None])
x_train, x_val, t_train, t_val = train_test_split(x_sc, t_sc, train_size=N_train)
x_train, x_val, t_train, t_val = (
    x_train.reshape(-1),
    x_val.reshape(-1),
    t_train.reshape(-1),
    t_val.reshape(-1),
)

# get features
Phi_plot = RadialBasisFunctions(
    xscaler.transform(x_plot[:, None]).reshape(-1), M_radial=M_radial, l_radial=l_radial
)
Phi_train = RadialBasisFunctions(x_train, M_radial=M_radial, l_radial=l_radial)
Phi_val = RadialBasisFunctions(x_val, M_radial=M_radial, l_radial=l_radial)

# create figure
fig, ax = plt.subplots(2, 2, figsize=(11, 8), sharey="row")
fig.canvas.toolbar_visible = False
plt.subplots_adjust(wspace=0.1)
ax[0, 0].plot(
    xscaler.inverse_transform(x_train[:, None]),
    tscaler.inverse_transform(t_train[:, None]),
    "x",
)
ax[0, 1].plot(
    xscaler.inverse_transform(x_train[:, None]),
    tscaler.inverse_transform(t_train[:, None]),
    "x",
)
ax[0, 0].plot(x_plot, f_truth(x_plot), "k-"), ax[0, 1].plot(
    x_plot, f_truth(x_plot), "k-"
)
ax[0, 0].set_xlabel("x"), ax[0, 1].set_xlabel("x"), ax[0, 0].set_ylabel("t")
ax[1, 0].set_xlabel("epochs"), ax[1, 1].set_xlabel("epochs"), ax[1, 0].set_ylabel(
    "RMSE"
)
ax[0, 0].set_ylim(-2, 2), ax[0, 1].set_ylim(-2, 2)
ax[0, 0].set_title("Unregularized"), ax[0, 1].set_title("Regularized")

# init rmse
rmse_train, rmse_train_reg = np.zeros(epochs), np.zeros(epochs)
rmse_val, rmse_val_reg = np.zeros(epochs), np.zeros(epochs)

# loop over epochs
for i in range(epochs):
    perm = np.random.permutation(N_train)

    # loop over batches
    for j in range(int(N_train / N_batch)):
        x_batch = x_train[perm[j * N_batch : (j + 1) * N_batch]]
        t_batch = t_train[perm[j * N_batch : (j + 1) * N_batch]]
        dEdw = get_gradient(
            x_batch,
            t_batch,
            w,
            RadialBasisFunctions,
            M_radial=M_radial,
            l_radial=l_radial,
        )
        dEdw_reg = get_gradient(
            x_batch,
            t_batch,
            w_reg,
            RadialBasisFunctions,
            lam=lam,
            M_radial=M_radial,
            l_radial=l_radial,
        )
        w = w - eta * dEdw
        w_reg = w_reg - eta * dEdw_reg

    # get predictions
    t_pred_train = w @ Phi_train.T
    t_pred_train_reg = w_reg @ Phi_train.T
    t_pred_val = w @ Phi_val.T
    t_pred_val_reg = w_reg @ Phi_val.T

    # compute rmse
    rmse_train[i] = get_rmse(t_train, t_pred_train, tscaler)
    rmse_train_reg[i] = get_rmse(t_train, t_pred_train_reg, tscaler)
    rmse_val[i] = get_rmse(t_val, t_pred_val, tscaler)
    rmse_val_reg[i] = get_rmse(t_val, t_pred_val_reg, tscaler)

    # plot some of the iterations to see progression of our model
    if (i + 1) in epoch_plot_marks:
        t_plot = w @ Phi_plot.T
        t_plot_reg = w_reg @ Phi_plot.T

        color = rgb[np.argwhere(epoch_plot_marks == i + 1)[0]]
        ax[0, 0].plot(
            x_plot,
            tscaler.inverse_transform(t_plot[:, None]).reshape(-1),
            color=color,
            label=r"{}".format(i + 1),
        )
        ax[0, 1].plot(
            x_plot,
            tscaler.inverse_transform(t_plot_reg[:, None]).reshape(-1),
            color=color,
            label=r"{}".format(i + 1),
        )

# put legend
ax[0, 0].legend(loc="lower left", ncol=2)
ax[0, 0].get_legend().set_title(r"epochs")

# plot rmse
ax[1, 0].semilogx(np.arange(1, epochs + 1), rmse_train, label=r"$RMSE_{train}$")
ax[1, 0].semilogx(np.arange(1, epochs + 1), rmse_val, label=r"$RMSE_{val}$")

ax[1, 1].semilogx(np.arange(1, epochs + 1), rmse_train_reg, label=r"$RMSE_{train}$")
ax[1, 1].semilogx(np.arange(1, epochs + 1), rmse_val_reg, label=r"$RMSE_{val}$")
ax[1, 0].legend(), ax[1, 1].legend()

plt.show()

```{figure} https://files.mude.citg.tudelft.nl/fig17.png
Training process of two models with different levels of regularization, predictions coming from model snapshots at different epochs.
```

You can see in the top figure that our predictions seem to converge towards a particular shape. The remaining discrepancy between our final model and the true function $f$ is due to our general model bias, and the particular dataset we drew.

You can see that our validation error increases at some point, indicating that overfitting might occur. Again, note how the training error cannot feel that, and just decreases monotonically. SGD with minibatches already has a slight regularizing effect. Other remedies include the L2-regularization technique discussed discusses previously, early stopping, or collecting more data.

Finally, it should be noted that the step size of the SGD must be chosen carefully. Try out for yourself what happens when you choose very small or large stepsizes by adapting the learning rate. Even though this optimization problem is well-defined and has a global minimum, SGD is not guaranteed to converge to it. Luckily, the most popular variants such as [AdaGrad][1], [RMSProp][2], and [Adam][3] feature some form of adaptive stepsize control, improving convergence rate and robustness. One usually starts with a larger stepsize to approach the minimum quickly. After that, the stepsize is reduced continuously to reliably uncover the exact location of the extremum.

[1]: https://jmlr.org/papers/volume12/duchi11a/duchi11a.pdf
[2]: http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
[3]: https://arxiv.org/abs/1412.6980

In the following animation you can see another example of extreme overfitting being observed during SGD iterations:

<center><img src="https://surfdrive.surf.nl/files/index.php/s/4Q8RDBb6ZKRgRBs/download" width="1000"></center>

Observing when the validation loss starts to increase is a useful sign of the onset of overfitting. Stopping the training process when this is detected is the core of so-called **early stopping** strategies implemented in popular machine learning packages.

% START-CREDIT
% source: machine_learning
```{attributiongrey} Attribution
:class: attribution
This chapter is written by Iuri Rocha, Anne Poot, Joep Storm and Leon Riccius. {ref}`Find out more here <machine_learning_credit>`.
```
% END-CREDIT